### Setup libraries

In [9]:
# A dependency of the preprocessing for BERT inputs
# !pip install tf-nightly
# !pip install -q -U tensorflow-text-nightly

In [2]:
# !pip install -q tf-models-official
# !pip install tf-models-official

In [3]:
# !git clone https://github.com/tensorflow/models.git

### Import libraries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:72.5% !important; }</style>"))

In [2]:
import os
import shutil
import re

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tqdm.notebook import tqdm
# from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

In [3]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

### Paths to directories and files

In [4]:
DRAFT_DATASET_PATH = 'D:/kaggle/mailru/Otvety.txt'
TRAIN_DATASET_PATH = 'D:/kaggle/mailru/answers.txt'

### Loading data

In [5]:
# # Small preprocess of the answers
# question = None
# written = False

# with open(TRAIN_DATASET_PATH, 'w', encoding='utf-8') as fout:
#     with open(DRAFT_DATASET_PATH, 'r', encoding='utf-8') as fin:
#         for line in tqdm(fin):
#             if line.startswith('---'):
#                 written = False
#                 continue
#             if not written and question is not None:
#                 fout.write(question.replace('\t', ' ').strip() + '\t' + line.replace('\t', ' '))
#                 written = True
#                 question = None
#                 continue
#             if not written:
#                 question = line.strip()
#                 continue

In [222]:
text = []

with open(DRAFT_DATASET_PATH, 'r', encoding='utf-8') as fin:
    for line in tqdm(fin):
        text.append(line)        

### Preprocessing data

In [223]:
TAG_RE = re.compile(r'<[^>]+>')
# TAG_RE = re.compile(r'<.*?>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [224]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
#     sentence = re.sub('[^А-Яа-я0-9,.!?-]', ' ', sentence)

    # Single character removal
#     sentence = re.sub(r'\s+[А-Яа-я0-9]\s+', ' ', sentence)

    # Removing multiple spaces
#     sentence = re.sub(r'\s+', ' ', sentence)

#     sentence = re.sub(':\)', '', sentence)
    sentence = re.sub('[)"]', '', sentence)  
    sentence = re.sub('\s*\?\s*\.', '?', sentence)
    sentence = re.sub('\s*\!\s*\.', '!', sentence)
    sentence = re.sub('\s*\.', '.', sentence)
    sentence = re.sub('\.+', '.', sentence)
#     sentence = re.sub('\s*\'\s*\.', ' ', sentence)

    
    return sentence

In [225]:
# Preprocess for models fitting
sentences = []

c = 0

with open(DRAFT_DATASET_PATH, 'r', encoding='utf-8') as fin:
    for line in tqdm(fin):
        spls = preprocess_text(line)
        sentences.append(spls)
        c += 1
        if c > 500:
            break

In [226]:
sentences[:100]

['\n',
 '---\n',
 'вопрос о ТДВ давно и хорошо отдыхаем ЛИЧНО ВАМ здесь кого советовали завести? \n',
 'хомячка. \n',
 'мужика, йопаря, собачку и 50 кошек. \n',
 'Общение! \n',
 'паучка. \n',
 'Да пол мне бы памыть!А таг то ни чо. Типа ни каво! \n',
 'я тут вообще что бы пообщаться. \n',
 'А мне советовали сиси завести. \n',
 'Ну, слава богу, мужика завести ещё не советовали А вот сватать к кому только не сватали. \n',
 'мне тут советовали завести любовника, мужа и много кошек приветик. \n',
 '---\n',
 'Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые. \n',
 'меня вобще прикалывает эта тема :. \n',
 'когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы. \n',
 '---\n',
 'Что делать, сегодня нашёл 2 миллиона рублей? \n',
 'Если это счастье  действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, 

### Sentenses

In [175]:
def split_by_sentence(sent):
    sent = sent.split('\n---\n')[1:]
    
    inp = []
    tar = []

    for se in sent:
        se = se.split('\n')
        inp.append(se[0].strip())
        tar.append(' '.join([f'<start> {s}<stop>' for s in se[1:]]))
    
    return inp, tar

In [176]:
inp, tar = split_by_sentence(sentences)
inp[0], tar[0]

AttributeError: 'list' object has no attribute 'split'

https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/

https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/

https://www.kaggle.com/nielspace/text-classification-using-bert